<a href="https://colab.research.google.com/github/ThevinduS/cuckoo/blob/master/LSTM3new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing liabraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install keras
!pip install tensorflow
#Install related libraries for the course.
#This is a common requirement for all other exampels too

!pip install pandas
!pip install tensorflow
!pip install sklearn
!pip install matplotlib
!pip install comet_ml

  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post7-py3-none-any.whl size=2950 sha256=6df8d297e7c9f22fbb2897ad5b132e60a22f4fe3a672109625a7f68389d1f7a2
  Stored in directory: /root/.cache/pip/wheels/c8/9c/85/72901eb50bc4bc6e3b2629378d172384ea3dfd19759c77fd2c
Successfully built sklearn
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.7/214.7 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.1/510.1 kB 24.4 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.6.1
    Uninst

In [3]:
# import comet_ml at the top of your file
from comet_ml import Experiment

# Create an experiment with your api key
experiment = Experiment(log_code=True,
    api_key="KgERfs0s0QEiRrMhOJYwGn5pk",
    project_name="dynamic-malware-analysis",
    workspace="thevindus",
)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/thevindus/dynamic-malware-analysis/682fa5ead23c456aa11d9087fe2aecc0



In [4]:
%load_ext autoreload

In [5]:
%autoreload 2
#import autoreload
#?autoreload


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import pickle as pickle

import random
random.seed(42)


from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score

from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.layers import Dense
import tensorflow as tf
print(tf.__version__)

2.12.0


Loading Dataset

In [6]:
malware_API_data = pd.read_excel("/content/drive/MyDrive/LSTM3/malware_API_dataset.xlsx", header = None, dtype= str)

In [7]:
dataset = pd.concat([malware_API_data], ignore_index=True)

In [8]:
from sklearn.utils import shuffle
dataset = shuffle(dataset)


In [9]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,588,589,590,591,592,593,594,595,596,597
16040,Trojan,UDS:DangerousObject.Multi.Generic,1ecc5004eecac737824fa66b361fd95534ff8138dae864...,GetThreadLocale,lstrcmpiW,GlobalAlloc,GlobalFree,FindResourceExW,LoadResource,LoadStringW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11877,Trojan,Trojan-FakeAV.Win32.Agent.dsl,e57876460ebd8b00e3fe159d78cb35876f9a41089c1a11...,lstrcpyW,GetThreadLocale,lstrcmpiW,GlobalAlloc,GlobalLock,GlobalUnlock,GlobalReAlloc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14649,Trojan.PSW,Trojan-PSW.Win32.Tepfer.baft,b39da71f54be10cbe0167c5ca4655831790c8f2d68eca5...,lstrcpyW,GetThreadLocale,lstrcmpiW,GlobalAlloc,GlobalLock,GlobalUnlock,GlobalReAlloc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15126,Trojan,Trojan-Ransom.Win32.Agent.hhk,e155fb4e1a7aacf9ef5424aead64dc7c8f83ddaa571514...,lstrcpyW,GetThreadLocale,lstrcmpiW,GlobalAlloc,GlobalLock,GlobalUnlock,GlobalReAlloc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12147,Trojan,Trojan-FakeAV.Win32.Agent.dvc,12d614d530aedb595523e09538994f3acd162b9292f283...,lstrcpyW,GetThreadLocale,lstrcmpiW,GlobalAlloc,GlobalLock,GlobalUnlock,GlobalReAlloc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5664,AdWare,not-a-virus:AdWare.Win32.MegaSearch.am,0dd3c4be20a22f2a0c1b7271266562ad83d17f659fc48b...,GetSystemDirectoryA,GetVersionExW,IsDBCSLeadByte,CompareStringW,DisableThreadLibraryCalls,GetVersionExA,GetModuleHandleW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8255,Trojan,Packed.Win32.Krap.iu,7fbb1e8b5dd5e4766e8a2503ce8696319a83c4e41b1344...,GetWindowsDirectoryW,VirtualAllocEx,lstrcatW,CreateFileW,VirtualQueryEx,GetThreadLocale,lstrcmpiW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8590,Trojan,Trojan.Win32.Agentb.nii,142f43a0dcb94f3eaf9dd772b7d32f4c276abb630631b3...,GetModuleFileNameW,LoadLibraryExW,LoadLibraryExA,LoadLibraryA,GetCommandLineA,GetEnvironmentStrings,GetVersion,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11550,Trojan,Trojan-FakeAV.Win32.Agent.dnw,bd3262c9147578dee23645f9bbdb43290d810a934effce...,lstrcpyW,GetThreadLocale,lstrcmpiW,GlobalAlloc,GlobalLock,GlobalUnlock,GlobalReAlloc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
print(dataset)

              0                                       1    \
16040      Trojan       UDS:DangerousObject.Multi.Generic   
11877      Trojan           Trojan-FakeAV.Win32.Agent.dsl   
14649  Trojan.PSW            Trojan-PSW.Win32.Tepfer.baft   
15126      Trojan           Trojan-Ransom.Win32.Agent.hhk   
12147      Trojan           Trojan-FakeAV.Win32.Agent.dvc   
...           ...                                     ...   
5664       AdWare  not-a-virus:AdWare.Win32.MegaSearch.am   
8255       Trojan                    Packed.Win32.Krap.iu   
8590       Trojan                 Trojan.Win32.Agentb.nii   
11550      Trojan           Trojan-FakeAV.Win32.Agent.dnw   
1100       Trojan               HEUR:Trojan.Win32.Generic   

                                                     2    \
16040  1ecc5004eecac737824fa66b361fd95534ff8138dae864...   
11877  e57876460ebd8b00e3fe159d78cb35876f9a41089c1a11...   
14649  b39da71f54be10cbe0167c5ca4655831790c8f2d68eca5...   
15126  e155fb4e1a7aacf9ef54

In [11]:
dataset_X = dataset.values[:,3:598]
dataset_X

array([['GetThreadLocale', 'lstrcmpiW', 'GlobalAlloc', ..., nan, nan,
        nan],
       ['lstrcpyW', 'GetThreadLocale', 'lstrcmpiW', ..., nan, nan, nan],
       ['lstrcpyW', 'GetThreadLocale', 'lstrcmpiW', ..., nan, nan, nan],
       ...,
       ['GetModuleFileNameW', 'LoadLibraryExW', 'LoadLibraryExA', ...,
        nan, nan, nan],
       ['lstrcpyW', 'GetThreadLocale', 'lstrcmpiW', ..., nan, nan, nan],
       ['FindResourceExW', 'LoadResource', 'GetProcessWindowStation',
        ..., nan, nan, nan]], dtype=object)

In [12]:
dataset_Y = dataset.values[:,0]
dataset_Y
#dataset.values[5000,0]

array(['Trojan', 'Trojan', 'Trojan.PSW', ..., 'Trojan', 'Trojan',
       'Trojan'], dtype=object)

Preprocessing data

In [13]:
le = preprocessing.LabelEncoder()

dataset_X_pandas = pd.DataFrame(dataset_X,dtype=str)
dataset_X_pandas.apply(le.fit_transform)

dataset_X = dataset_X_pandas.apply(le.fit_transform).values[:,:]
dataset_X

array([[ 58, 142,  90, ...,  25,  15,  10],
       [125,  76, 149, ...,  25,  15,  10],
       [125,  76, 149, ...,  25,  15,  10],
       ...,
       [ 41, 104, 103, ...,  25,  15,  10],
       [125,  76, 149, ...,  25,  15,  10],
       [ 18, 105,  65, ...,  25,  15,  10]])

In [14]:
le = preprocessing.LabelEncoder()

dataset_Y_pandas = pd.DataFrame(dataset_Y,dtype=str)
dataset_Y_pandas.apply(le.fit_transform)

dataset_Y = dataset_Y_pandas.apply(le.fit_transform).values[:,:]

dataset_Y

array([[6],
       [6],
       [7],
       ...,
       [6],
       [6],
       [6]])

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(dataset_X, dataset_Y, test_size=0.5, random_state=42)

In [16]:
from sklearn.preprocessing import MinMaxScaler

data = X_train
scaler = MinMaxScaler()
print(scaler.fit(data))

print(scaler.transform(data))

X_train = scaler.transform(data)

X_test = scaler.transform(X_test)

MinMaxScaler()
[[0.65354331 0.70068027 0.40397351 ... 1.         1.         1.        ]
 [0.97637795 0.5170068  0.97350993 ... 1.         1.         1.        ]
 [0.48818898 0.55782313 0.39072848 ... 1.         1.         1.        ]
 ...
 [0.48818898 0.55782313 0.39072848 ... 1.         1.         1.        ]
 [0.48818898 0.55782313 0.39072848 ... 1.         1.         1.        ]
 [0.4015748  0.56462585 0.60927152 ... 1.         1.         1.        ]]


In [17]:
from sklearn.preprocessing import MinMaxScaler

data = Y_train
scaler = MinMaxScaler()
print(scaler.fit(data))

print(scaler.transform(data))

Y_train = scaler.transform(data)
Y_test = scaler.transform(Y_test)

MinMaxScaler()
[[0.81818182]
 [0.54545455]
 [0.        ]
 ...
 [0.54545455]
 [0.        ]
 [0.54545455]]


In [18]:
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(Y_train,num_classes=12)  # Assuming Y
Y_test = to_categorical(Y_test,num_classes=12)  #

LSTM model

In [19]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

In [20]:
vocab_size = 1000
max_sequence_length = 595

In [21]:
model = Sequential()
model.add(Embedding(vocab_size, 32, input_length=max_sequence_length))
model.add(LSTM(64))
model.add(Dense(12, activation='softmax'))
model.summary()

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 595, 32)           32000     
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 12)                780       
                                                                 
Total params: 57,612
Trainable params: 57,612
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(X_train, Y_train, batch_size=32, epochs=20,validation_split=0.2)


COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE


Epoch 1/20
216/216 [==============================] - 89s 394ms/step - loss: 0.1643 - accuracy: 0.9952 - val_loss: 0.0159 - val_accuracy: 0.9983
Epoch 2/20
216/216 [==============================] - 83s 385ms/step - loss: 0.0017 - accuracy: 0.9999 - val_loss: 0.0159 - val_accuracy: 0.9983
Epoch 3/20
216/216 [==============================] - 81s 375ms/step - loss: 0.0016 - accuracy: 0.9999 - val_loss: 0.0162 - val_accuracy: 0.9983
Epoch 4/20
216/216 [==============================] - 85s 394ms/step - loss: 0.0016 - accuracy: 0.9999 - val_loss: 0.0158 - val_accuracy: 0.9983
Epoch 5/20
216/216 [==============================] - 82s 378ms/step - loss: 0.0016 - accuracy: 0.9999 - val_loss: 0.0162 - val_accuracy: 0.9983
Epoch 6/20
216/216 [==============================] - 83s 384ms/step - loss: 0.0016 - accuracy: 0.9999 - val_loss: 0.0153 - val_accuracy: 0.9983
Epoch 7/20
216/216 [==============================] - 80s 370ms/step - loss: 0.0015 - accuracy: 0.9999 - val_loss: 0.0150 - val_ac

In [23]:
# Save the model
#model.save("E:\\NSBM\\Year 3\\Semester 1\\PUSL3119 Computing Project\\Projects\\Anti-Spy Application\\Ai algorithms\\Trained algorithms\\LSTM 3")


In [24]:
score = model.evaluate(X_test, Y_test)

270/270 [==============================] - 24s 86ms/step - loss: 0.0055 - accuracy: 0.9994


In [ ]:
score

LSTM model advance

In [26]:
from comet_ml import Experiment

# Create an experiment with your api key
experiment = Experiment(log_code=True,
    api_key="KgERfs0s0QEiRrMhOJYwGn5pk",
    project_name="dynamic-malware-analysis",
    workspace="thevindus",
)

COMET WARNING: Couldn't retrieve Google Colab notebook content
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/thevindus/dynamic-malware-analysis/682fa5ead23c456aa11d9087fe2aecc0
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [20]                 : (0.9952236413955688, 0.9998552799224854)
COMET INFO:     batch_accuracy [440]          : (0.0, 1.0)
COMET INFO:     batch_loss [440]              : (5.8530044043436646e-05, 2.493279457092285)
COMET INFO:     epoch_duration [20]           : (79.71029320399998, 88.73222271899999)
COMET INFO:     loss [20]                     : (0.0014652252430096269, 0.16431495547294617)
COMET INFO:     val_accuracy               

In [27]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding,Dropout

In [28]:
vocab_size = 1000
max_sequence_length = 595

In [29]:
model = Sequential()
model.add(Embedding(vocab_size, 32, input_length=max_sequence_length))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(12, activation='softmax'))
model.summary()

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 595, 32)           32000     
                                                                 
 lstm_1 (LSTM)               (None, 64)                24832     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 12)                780       
                                                                 
Total params: 57,612
Trainable params: 57,612
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.fit(X_train, Y_train, batch_size=32, epochs=20,validation_split=0.2)

Epoch 1/20
216/216 [==============================] - 84s 378ms/step - loss: 0.1829 - accuracy: 0.9849 - val_loss: 0.0169 - val_accuracy: 0.9983
Epoch 2/20
216/216 [==============================] - 81s 376ms/step - loss: 0.0021 - accuracy: 0.9999 - val_loss: 0.0175 - val_accuracy: 0.9983
Epoch 3/20
216/216 [==============================] - 80s 370ms/step - loss: 0.0018 - accuracy: 0.9999 - val_loss: 0.0176 - val_accuracy: 0.9983
Epoch 4/20
216/216 [==============================] - 83s 382ms/step - loss: 0.0017 - accuracy: 0.9999 - val_loss: 0.0175 - val_accuracy: 0.9983
Epoch 5/20
216/216 [==============================] - 81s 376ms/step - loss: 0.0019 - accuracy: 0.9999 - val_loss: 0.0174 - val_accuracy: 0.9983
Epoch 6/20
216/216 [==============================] - 83s 383ms/step - loss: 0.0018 - accuracy: 0.9999 - val_loss: 0.0169 - val_accuracy: 0.9983
Epoch 7/20
216/216 [==============================] - 80s 372ms/step - loss: 0.0017 - accuracy: 0.9999 - val_loss: 0.0180 - val_ac

In [31]:
score = model.evaluate(X_test, Y_test)

270/270 [==============================] - 23s 84ms/step - loss: 0.0056 - accuracy: 0.9994


In [ ]:
score

MLP model

In [33]:
from comet_ml import Experiment

# Create an experiment with your api key
experiment = Experiment(log_code=True,
    api_key="KgERfs0s0QEiRrMhOJYwGn5pk",
    project_name="dynamic-malware-analysis",
    workspace="thevindus",
)

COMET WARNING: Couldn't retrieve Google Colab notebook content
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/thevindus/dynamic-malware-analysis/585c0469ce6a4e38b78e56198a0cae93
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [20]                 : (0.984947144985199, 0.9998552799224854)
COMET INFO:     batch_accuracy [440]          : (0.0, 1.0)
COMET INFO:     batch_loss [440]              : (9.113082342082635e-05, 2.499173402786255)
COMET INFO:     epoch_duration [20]           : (79.91336319999982, 83.90927074200022)
COMET INFO:     loss [20]                     : (0.0014165266184136271, 0.18292038142681122)
COMET INFO:     val_accuracy                 

In [35]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense

# Assuming you have your training data ready: x_train (sequences) and y_train (labels)

vocab_size = 1000  # Change this value to the actual size of your vocabulary
max_sequence_length = 595 # Change this value to the actual maximum sequence length

model_mlp = Sequential()
model_mlp.add(Embedding(vocab_size, 32, input_length=max_sequence_length))
model_mlp.add(Flatten())
model_mlp.add(Dense(128, activation='relu'))
model_mlp.add(Dense(12, activation='softmax'))

model_mlp.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_mlp.fit(X_train, Y_train, epochs=20, batch_size=32)

score = model_mlp.evaluate(X_test, Y_test)
score


Epoch 1/20
270/270 [==============================] - 12s 41ms/step - loss: 0.0206 - accuracy: 0.9958
Epoch 2/20
270/270 [==============================] - 11s 40ms/step - loss: 0.0060 - accuracy: 0.9995
Epoch 3/20
270/270 [==============================] - 9s 33ms/step - loss: 0.0046 - accuracy: 0.9995
Epoch 4/20
270/270 [==============================] - 11s 40ms/step - loss: 0.0052 - accuracy: 0.9995
Epoch 5/20
270/270 [==============================] - 11s 40ms/step - loss: 0.0053 - accuracy: 0.9995
Epoch 6/20
270/270 [==============================] - 9s 33ms/step - loss: 0.0050 - accuracy: 0.9995
Epoch 7/20
270/270 [==============================] - 11s 40ms/step - loss: 0.0049 - accuracy: 0.9995
Epoch 8/20
270/270 [==============================] - 11s 40ms/step - loss: 0.0065 - accuracy: 0.9995
Epoch 9/20
270/270 [==============================] - 10s 35ms/step - loss: 0.0063 - accuracy: 0.9995
Epoch 10/20
270/270 [==============================] - 11s 39ms/step - loss: 0.0050 

[0.0064275553449988365, 0.9994211792945862]

GRU model

In [36]:
from comet_ml import Experiment

# Create an experiment with your api key
experiment = Experiment(log_code=True,
    api_key="KgERfs0s0QEiRrMhOJYwGn5pk",
    project_name="dynamic-malware-analysis",
    workspace="thevindus",
)

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/thevindus/dynamic-malware-analysis/c9efa87656544dd9a7b76b14836a5cca
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [20]        : (0.9958319067955017, 0.9995368719100952)
COMET INFO:     batch_accuracy [540] : (0.0, 1.0)
COMET INFO:     batch_loss [540]     : (2.602082986413734e-06, 2.529268741607666)
COMET INFO:     epoch_duration [20]  : (8.802445921000071, 11.756489011999747)
COMET INFO:     loss [20]            : (0.003900384297594428, 0.020619940012693405)
COMET INFO:   Others:
COMET INFO:     trainable_params : 2470796
COMET INFO:   Parameters:
COMET INFO:     Adam_amsgrad                 : False
COMET I

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

# Assuming you have your preprocessed data ready: x_train, y_train, x_test, y_test

vocab_size = 1000  # Replace this with the actual size of your vocabulary
max_sequence_length = 595  # Replace this with the actual maximum sequence length

gru_model = Sequential()
gru_model.add(Embedding(vocab_size, 32, input_length=max_sequence_length))
gru_model.add(GRU(64))  # You can adjust the number of GRU units as needed
gru_model.add(Dense(64, activation='relu'))
gru_model.add(Dense(12, activation='softmax'))  # Assuming you have 11 classes for malware categorization



# Train the GRU model
gru_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

gru_model.fit(X_train, Y_train, epochs=20, batch_size=32)

score = model_mlp.evaluate(X_test, Y_test)
score



Epoch 1/20
270/270 [==============================] - 94s 332ms/step - loss: 0.1878 - accuracy: 0.9921
Epoch 2/20
270/270 [==============================] - 90s 332ms/step - loss: 0.0060 - accuracy: 0.9995
Epoch 3/20
270/270 [==============================] - 92s 339ms/step - loss: 0.0047 - accuracy: 0.9995
Epoch 4/20
270/270 [==============================] - 87s 322ms/step - loss: 0.0044 - accuracy: 0.9995
Epoch 5/20
270/270 [==============================] - 87s 322ms/step - loss: 0.0044 - accuracy: 0.9995
Epoch 6/20
270/270 [==============================] - 90s 332ms/step - loss: 0.0043 - accuracy: 0.9995
Epoch 7/20
270/270 [==============================] - 90s 332ms/step - loss: 0.0043 - accuracy: 0.9995
Epoch 8/20
270/270 [==============================] - 93s 346ms/step - loss: 0.0041 - accuracy: 0.9995
Epoch 9/20
270/270 [==============================] - 90s 334ms/step - loss: 0.0044 - accuracy: 0.9995
Epoch 10/20
270/270 [==============================] - 91s 338ms/step - l

[0.0064275553449988365, 0.9994211792945862]